In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

--2020-05-16 10:59:21--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

100%[======================================>] 23,101      --.-K/s   in 0.001s  

2020-05-16 10:59:21 (18.8 MB/s) - ‘loan_train.csv’ saved [23101/23101]



In [ ]:
df = pd.read_csv('loan_train.csv')

df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df['dayofweek'] = df['effective_date'].dt.dayofweek
df['weekend']= df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
# Feature.drop(['Master or Above'], axis = 1,inplace=True)
df['loan_status'].replace(to_replace=['PAIDOFF','COLLECTION'], value=[0,1],inplace=True)


Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)

X = Feature
y = df.loan_status

X = preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

pen = ['l1', 'l2']
C = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]
grid_vals = {'penalty': pen, 'C': C}

lr = LogisticRegression()
grid_lr = GridSearchCV(lr, param_grid=grid_vals).fit(X, y)
best_params_lr = grid_lr.best_params_
# 


In [57]:
from sklearn.neighbors import KNeighborsClassifier
neighbors = range(1,50)
grid_vals = {'n_neighbors': neighbors}
knn = KNeighborsClassifier()

grid_knn = GridSearchCV(knn, param_grid=grid_vals).fit(X, y)
best_params_knn = grid_knn.best_params_



/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'n_neighbors': 40}

In [64]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
criterion = ['gini', 'entropy']
max_depth = range(1,20)
min_samples_leaf = range(1,20)
grid_vals = {'criterion': criterion, 'max_depth': max_depth, 'min_samples_leaf': min_samples_leaf}

grid_dt = GridSearchCV(dt, param_grid=grid_vals).fit(X, y)
best_params_dt = grid_dt.best_params_

best_params_dt

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 1}

In [73]:
from sklearn.svm import SVC 
svm = SVC() 
grid_vals = [{'C': range(1,100),  
              'gamma': range(1,100) ,
              'kernel': ['rbf']}, {'kernel': ['linear'], 'C': range(1,100)}]

grid_svm = GridSearchCV(svm, param_grid=grid_vals).fit(X, y)
best_params_svm = grid_svm.best_params_

best_params_svm


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 1, 'gamma': 25, 'kernel': 'rbf'}

NameError: name 'loguniform' is not defined

In [56]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
test_df = pd.read_csv('loan_test.csv')
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_Feature = test_df[['Principal','terms','age','Gender','weekend']]
test_Feature = pd.concat([test_Feature,pd.get_dummies(test_df['education'])], axis=1)
test_Feature.drop(['Master or Above'], axis = 1,inplace=True)
test_df['loan_status'].replace(to_replace=['PAIDOFF','COLLECTION'], value=[0,1],inplace=True)
test_y = test_df['loan_status'].values

test_X = preprocessing.StandardScaler().fit(Feature).transform(test_Feature)
LR_model = LogisticRegression(C= best_params["C"], penalty= best_params["penalty"]).fit(X,y)

LR_yhat = LR_model.predict(test_X)
LR_yhat_prob = LR_model.predict_proba(test_X)
print("LR Jaccard index: %.2f" % jaccard_similarity_score(test_y, LR_yhat))
print("LR F1-score: %.2f" % f1_score(test_y, LR_yhat, average='weighted') )
print("LR LogLoss: %.2f" % log_loss(test_y, LR_yhat_prob))
    

--2020-05-16 12:50:26--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3642 (3.6K) [text/csv]
Saving to: ‘loan_test.csv’

100%[======================================>] 3,642       --.-K/s   in 0s      

2020-05-16 12:50:26 (291 MB/s) - ‘loan_test.csv’ saved [3642/3642]

LR Jaccard index: 0.74
LR F1-score: 0.63
LR LogLoss: 0.69


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:17: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [62]:
KNN_model = KNeighborsClassifier(n_neighbors=best_params_knn["n_neighbors"]).fit(X,y)

KNN_yhat = KNN_model.predict(test_X)
print("LR Jaccard index: %.2f" % jaccard_similarity_score(test_y, KNN_yhat))
print("LR F1-score: %.2f" % f1_score(test_y, KNN_yhat, average='weighted') )
    

LR Jaccard index: 0.80
LR F1-score: 0.74
LR LogLoss: 7.04


In [65]:
DT_model = DecisionTreeClassifier(criterion = best_params_dt["criterion"], max_depth = best_params_dt["max_depth"] ,min_samples_leaf = best_params_dt["min_samples_leaf"] ).fit(X,y)
DT_yhat = DT_model.predict(test_X)
print("LR Jaccard index: %.2f" % jaccard_similarity_score(test_y, DT_yhat))
print("LR F1-score: %.2f" % f1_score(test_y, DT_yhat, average='weighted') )
    

LR Jaccard index: 0.74
LR F1-score: 0.63


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
svm_model = SVC(C= best_params_svm["C"], gamma= best_params_svm["gamma"], kernel= best_params_svm["kernel"]).fit(X,y)

svm_yhat = svm_model.predict(test_X)
print("LR Jaccard index: %.2f" % jaccard_similarity_score(test_y, svm_yhat))
print("LR F1-score: %.2f" % f1_score(test_y, svm_yhat, average='weighted') )


LR Jaccard index: 0.81
LR F1-score: 0.77


In [60]:
best_params_knn('n_neighbors')

TypeError: 'dict' object is not callable

In [54]:
best_params["C"]

0.001